In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import yfinance as yf
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

# Define the ticker symbol
ticker_symbol = 'SPY'

# Download historical data from Yahoo Finance
data = yf.download(ticker_symbol, start='2008-01-01', end='2011-01-01')

# Select only the 'Close' prices
close_prices = data['Close']

ret = np.log(1+close_prices.pct_change())
# Convert the Series to a DataFrame
df = pd.DataFrame(close_prices)



# In[2]:


# Print the DataFrame
print(df)


# In[3]:


plt.plot(df)


# In[4]:


ret


# In[5]:


mean = ret.mean()
std = ret.std()

# Print the mean and variance
print("Mean:")
print(mean)
print("\nStandard deviation:")
print(std)


# In[6]:


ret.shape[0]


# In[7]:


simulated_data=np.random.normal(mean,std,ret.shape[0])


# In[8]:


starting_stock_price=400
sim_stock_price=starting_stock_price*(simulated_data+1).cumprod()


# In[9]:


sim_stock_price=pd.DataFrame(sim_stock_price)


# In[10]:


sim_stock_price


# In[11]:


sim_stock_price.index=df.index


# In[12]:


sim_stock_price


# In[13]:


df


# In[14]:


plt.plot(sim_stock_price)


# In[15]:


simulations_mc=[]
for i in range(1000):
    simulated_data=np.random.normal(mean,std,ret.shape[0])
    starting_stock_price=400
    sim_stock_price=starting_stock_price*(simulated_data+1).cumprod()
    plt.axhline(starting_stock_price,c='k')
    plt.plot(sim_stock_price)
    df_mc = pd.DataFrame(sim_stock_price, columns=['Price'])
    simulations_mc.append(df_mc)


# In[22]:


simulations_mc[6]


# # Monte Carlo models
# ### Assumptions of Geometric Brownian Motion(Constant mean and Variane)
# ### X(t) = X(0) * e^((μ - (σ^2)/2) * t + σ * B(t))
# 
# ## Assumes  logarithmic returns of the asset follow a normal distribution and asset's price evolves over time.
# 
# ### Strenghts and Weakness
# 
# ### expected returns, volatility, scenorias
# 

# In[38]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parameters
mu = mean
n = 1000  # Number of intervals
T = 4  # Time in years
M = 1000  # Number of simulations
S0 = 400  # Initial stock price
sigma = std  # Volatility

# Calculate each time step
dt = T / n

# Simulation using numpy arrays
np.random.seed(42)
St = np.exp(
    (mu - sigma ** 2 / 2) * dt
    + sigma * np.random.normal(0, np.sqrt(dt), size=(M, n)).T
)

# Multiply through by S0 and return the cumulative product of elements along a given simulation path (axis=0).
St = S0 * St.cumprod(axis=0)

# Store simulation data in separate dataframes
simulations_gbm = []
for i in range(M):
    df = pd.DataFrame({"Stock Price": St[:, i]})
    simulations_gbm.append(df)

# Plot the simulated GBM realizations
plt.figure(figsize=(12, 6))
for i in range(M):
    plt.plot(simulations_gbm[i]["Stock Price"])

plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.title("Geometric Brownian Motion Simulation")
plt.show()


# In[41]:


simulations_gbm[6]


# # Heston Model
# ## GBM doesnt capture volatilty component
# ### VOlatility is not constant, mean reverting,

# In[19]:


import numpy as np
import matplotlib.pyplot as plt

# Define mean and standard deviation of returns



# Calculate parameters for the Heston model
kappa = 2  # Mean reversion speed of variance
theta = std ** 2  # Long-term average variance
sigma = std  # Volatility of volatility
rho = -0.5  # Correlation between the stock price and its volatility

# Heston model parameters
S0 = 400  # Initial stock price
r = 0.05  # Risk-free interest rate
T = 1  # Time to maturity (in years)
N = 860  # Number of time steps
dt = T / N  # Time increment
num_simulations = 100  # Number of simulations

# Simulate stock prices using the Heston model
#np.random.seed(42)
simulations_hm = []

plt.figure(figsize=(12, 6))

for i in range(num_simulations):
    V = np.zeros(N+1)
    V[0] = theta

    for t in range(1, N+1):
        #dz1 used to model the randomness or noise in the volatility process.
        dZ1 = np.random.normal(0, np.sqrt(dt))
        # generated to introduce correlation between the stock price and its volatility. 
        dZ2 = rho * dZ1 + np.sqrt(1 - rho**2) * np.random.normal(0, np.sqrt(dt))
        # generated to introduce correlation between the stock price and its volatility. mean reversion
        V[t] = V[t-1] + kappa * (theta - V[t-1]) * dt + sigma * np.sqrt(V[t-1]) * dZ1

    S = np.zeros(N+1)
    S[0] = S0

    for t in range(1, N+1):
        dW = np.random.normal(0, np.sqrt(dt))
        S[t] = S[t-1] * np.exp((r - 0.5 * V[t]) * dt + np.sqrt(V[t]) * dW)

    df_heston = pd.DataFrame(S, columns=['Price'])
    simulations_hm.append(df_heston)

    # Plot the simulated stock prices
    plt.plot(np.linspace(0, T, N+1), S)

plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.title('Simulated Stock Prices (Heston Model)')
plt.show()


# In[20]:


simulations_hm[0]


# In[ ]:




